# Working with uDALES Facet Data

This tutorial covers how to work with uDALES facet data, including surface quantities such as shear stress, surface pressure, surface energy balance terms, and temperature inside each of the facets.

The **UDBase** class provides comprehensive methods for facet analysis:
- **calculate_frontal_properties**: Calculate skylines, frontal areas, blockage ratios
- **plot_fac_type**: Display the type of surface for each facet
- **assign_prop_to_fac**: Assign properties to facets for calculation/visualization
- **plot_fac**: Display a surface variable on the mesh
- **load_fac_momentum**: Load facet pressure and shear stresses
- **load_fac_eb**: Load surface energy balance terms
- **load_seb**: Load all surface energy balance terms
- **load_fac_temperature**: Load temperatures inside facets
- **area_average_seb**: Area-averaged surface energy balance
- **area_average_fac**: Area-averaging over facets
- **time_average**: Time-averaging method (static)
- **convert_fac_to_field**: Convert facet data to 3D density field

## 1. Import Libraries and Setup

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

# Add the uDALES python path
sys.path.insert(0, '../tools/python')

from udbase import UDBase

# Create an instance of the UDBase class
expnr = '065'
expdir = '../experiments/065'
sim = UDBase(case_dir=expdir)

## 2. Calculate Frontal Properties

In [ ]:
help(sim.calculate_frontal_properties)

In [ ]:
# Calculate frontal properties
res = sim.calculate_frontal_properties()

print("X-direction:")
print(f"  Frontal area: {res['frontal_area_x']:.1f} m²")
print(f"  Blockage ratio: {res['blockage_ratio_x']:.3f}")
print()
print("Y-direction:")
print(f"  Frontal area: {res['frontal_area_y']:.1f} m²")
print(f"  Blockage ratio: {res['blockage_ratio_y']:.3f}")

## 3. Facet Types and Properties

Facet types contain properties for each wall type, including physical and thermal properties.

In [ ]:
# Assign albedo to facets
albs = sim.assign_prop_to_fac('al')
print(f"Albedo range: {albs.min():.3f} to {albs.max():.3f}")

# Visualize albedo on facets
sim.plot_fac(albs, title='Albedo')

## 4. Key Facet Methods Overview

### Visualization Methods
- `plot_fac_type()` - Display surface types (walls/roofs/ground)
- `plot_fac(data, title, cmap)` - Display variable on 3D mesh
- `assign_prop_to_fac(prop)` - Assign properties to individual facets

### Data Loading Methods
- `load_fac_momentum(var, time)` - Pressure, shear stress (fac.expnr.nc)
- `load_fac_eb(var, time)` - Individual SEB terms (facEB.expnr.nc)
- `load_seb(time)` - All SEB terms at once (recommended)
- `load_fac_temperature(time)` - Facet temperature (facT.expnr.nc)

### Analysis Methods
- `area_average_fac(data, areas)` - Area-weighted averaging
- `area_average_seb(seb_dict, facet_type)` - Average SEB by type
- `time_average(data, axis)` - Time averaging (static method)
- `convert_fac_to_field(fac_data, method)` - Convert to 3D field

## 5. Load Surface Energy Balance (SEB) Data

In [ ]:
help(sim.load_seb)

In [ ]:
# Load SEB at t=3600s
seb = sim.load_seb(time=3600)
print(f"SEB components: {list(seb.keys())}")

# Check energy balance
balance = seb['qnet'] - (seb['qsens'] + seb['qlat'] + seb['qstor'])
print(f"Max energy balance residual: {np.abs(balance).max():.2f} W/m²")

# Visualize sensible heat flux
sim.plot_fac(seb['qsens'], title='Sensible Heat Flux [W/m²]', cmap='RdBu_r')

## 6. Area-Averaged Surface Energy Balance

Calculate area-weighted averages for all SEB components, optionally by facet type.

In [ ]:
# Calculate area-averaged SEB by facet type
avg_all = sim.area_average_seb(seb)
avg_wall = sim.area_average_seb(seb, facet_type='wall')
avg_roof = sim.area_average_seb(seb, facet_type='roof')
avg_ground = sim.area_average_seb(seb, facet_type='ground')

# Display results
print("All surfaces:")
print(f"  Qsens: {avg_all['qsens']:6.2f} W/m²")
print(f"  Qlat:  {avg_all['qlat']:6.2f} W/m²")
print()
print("Walls:")
print(f"  Qsens: {avg_wall['qsens']:6.2f} W/m²")
print()
print("Roofs:")
print(f"  Qsens: {avg_roof['qsens']:6.2f} W/m²")
print()
print("Ground:")
print(f"  Qsens: {avg_ground['qsens']:6.2f} W/m²")

## 7. Complete Workflow Example

Typical facet analysis workflow from data loading to visualization and analysis.

In [ ]:
# Complete workflow example

# 1. Calculate urban metrics
frontal = sim.calculate_frontal_properties()
print(f"Blockage ratio: {frontal['blockage_ratio_x']:.3f}")

# 2. Load SEB data
seb = sim.load_seb(time=3600)

# 3. Calculate averages
avg_all = sim.area_average_seb(seb)
avg_wall = sim.area_average_seb(seb, facet_type='wall')

# 4. Check energy balance
balance = avg_all['qnet'] - (avg_all['qsens'] + avg_all['qlat'] + avg_all['qstor'])
print(f"Energy balance residual: {balance:.2f} W/m²")

# 5. Visualize all SEB components
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

sim.plot_fac(seb['qsens'], title='Sensible Heat Flux', ax=axes[0,0])
sim.plot_fac(seb['qlat'], title='Latent Heat Flux', ax=axes[0,1])
sim.plot_fac(seb['qnet'], title='Net Radiation', ax=axes[1,0])
sim.plot_fac(seb['qstor'], title='Storage Flux', ax=axes[1,1])

plt.tight_layout()

## 8. Summary

### Key Capabilities
- ✅ Load all facet data types (momentum, energy balance, temperature)
- ✅ Area-weighted averaging over facets with type-specific analysis
- ✅ 3D visualization of facet variables on building surfaces
- ✅ Convert facet data to 3D fields for volume analysis
- ✅ Calculate urban canopy metrics (frontal area, blockage ratio)

### Facet Data Sources
| Method | File | Variables |
|--------|------|-----------|
| `load_fac_momentum` | fac.expnr.nc | pres, tau_x/y/z, htc |
| `load_fac_eb` | facEB.expnr.nc | qsens, qlat, qnet, qstor |
| `load_seb` | facEB.expnr.nc | All SEB terms (recommended) |
| `load_fac_temperature` | facT.expnr.nc | Temperature profiles |

### Next Steps
- **geometry_tutorial.ipynb** - Creating and manipulating urban geometries
- `tools/python/facets_example.py` - Complete working examples
- `tools/python/advanced_analysis_example.py` - Advanced workflows
- uDALES documentation: https://u-dales.readthedocs.io/